In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import talib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
input_csv_file = ".csv"
output_csv_file = "_output.csv"

In [ ]:
df = pd.read_csv(input_csv_file)
df["open"] = df["current"] - df["change"]

In [ ]:
plt.plot(df["current"])

In [ ]:
df["time"] = pd.to_datetime(df["time"], format="%H:%M:%S").dt.time
start_time_1 = pd.Timestamp("09:30:00").time()
end_time_1 = pd.Timestamp("12:00:00").time()
start_time_2 = pd.Timestamp("13:00:00").time()
end_time_2 = pd.Timestamp("16:00:00").time()
filter_1 = (df["time"] >= start_time_1) & (df["time"] <= end_time_1)
filter_2 = (df["time"] >= start_time_2) & (df["time"] <= end_time_2)
filter_df = df[filter_1 | filter_2]
filter_df

In [ ]:
from datetime import datetime, timedelta

start_time = datetime.combine(datetime.today().date(), datetime.min.time())
# 1 week (9:30 - 12:00 & 13:00 - 16:00, 330 * 5 = 1650 minutes) => 5 minutes
# 1650 / 5 = 330
# send a request every 2 seconds (11 * 30 = 330) => send a request every 11 minutes
interval = timedelta(minutes=10)
time_set = set()
end_time = start_time + timedelta(days=1)
current_time = start_time
while current_time < end_time:
    time_set.add(current_time.time())
    current_time += interval

filter_df = filter_df[filter_df["time"].isin(time_set)]
filter_df

In [ ]:
del filter_df["date"]
del filter_df["time"]
del filter_df["status"]
del filter_df["change"]
data = filter_df.rename(
    columns={
        "current": "Close",
        "turnover": "Volume",
        "high": "High",
        "low": "Low",
        "percent": "Percent",
        "open": "Open",
    }
)
data = data.reindex(columns=["Open", "High", "Low", "Close", "Volume", "Percent"])
data

In [ ]:
data.to_csv(output_csv_file, index=False)

In [ ]:
data = data.assign(Predict=False)
top = data.nlargest(8, "Close")
data.loc[data.index.isin(top.index), "Predict"] = True

In [ ]:
data["RSI"] = talib.RSI(data["Close"].values, timeperiod=14)
data["SLOWK"], data["SLOWD"] = talib.STOCH(
    data["High"].values,
    data["Low"].values,
    data["Close"].values,
    fastk_period=5,
    slowk_period=3,
    slowk_matype=0,
    slowd_period=3,
    slowd_matype=0,
)
data["ATR"] = talib.ATR(
    data["High"].values, data["Low"].values, data["Close"].values, timeperiod=14
)
data["MOM"] = talib.MOM(data["Close"].values, timeperiod=10)
data["MFI"] = talib.MFI(
    data["High"].values,
    data["Low"].values,
    data["Close"].values,
    data["Volume"].values.astype(np.float64),
    timeperiod=14,
)
data["ROC"] = talib.ROC(data["Close"].values, timeperiod=10)
data["OBV"] = talib.OBV(data["Close"].values, data["Volume"].values.astype(np.float64))
data["CCI"] = talib.CCI(
    data["High"].values, data["Low"].values, data["Close"].values, timeperiod=14
)
data["TRIX"] = talib.TRIX(data["Close"].values, timeperiod=30)
data["EMA"] = talib.EMA(data["Close"].values, timeperiod=30)

macd, macdsignal, macdhist = talib.MACD(
    data["Close"].values, fastperiod=12, slowperiod=26, signalperiod=9
)
data["MACD"] = macd
data["MA"] = talib.MA(data["Close"].values, timeperiod=30, matype=0)

del data["Open"]
del data["High"]
del data["Low"]
del data["Close"]
del data["Volume"]
del data["Percent"]
data

In [ ]:
x = data[[col for col in data.columns if col != "Predict"]]
y = data["Predict"]
x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=0.75, random_state=10
)

model = RandomForestClassifier(n_jobs=-1, n_estimators=65, random_state=10)
model.fit(x_train, y_train)
prediction = model.predict(x_test)
report = classification_report(y_test, prediction)
print(report)